In [1]:
#INITIALIZE
using DSWB, Formatting
using DSWBIJuliaComm
include("./init.jl")

HTML{ASCIIString}("<script> (function() {\n    window.DSWB = window.DSWB || {};\n\n    //check if the setup has already been completed.\n    if (window.DSWB.listening) {\n        return;\n    }\n    window.DSWB.listening = true;\n\n    var allowedOrigins = {\n        \"https://localhost:8443\": true,\n        \"https://dswb-exp.soasta.com\": true,\n        \"https://dswb-dev.soasta.com\": true\n    };\n\n    //the current server is always an allowed origin.\n    allowedOrigins[window.location.protocol + \"//\" + window.location.host] = true;\n\n    //create empty queue to keep targetless messages\n    window.DSWB.msgQueue = window.DSWB.msgQueue || [];\n\n\n    window.addEventListener(\"message\", function(event) {\n\n        //discard invalid messages.\n        if (!allowedOrigins[event.origin] || !event.data) {\n            // console.log(\"no event data\");\n            return;\n        }\n        //check that IJulia's working.\n        if (isIJuliaBroken()) {\n            // console.log(\"ijulia is broken\");\n            return;\n        }\n        if (event.data.msg_type === \"forward\") {\n            var kernel = IPython.notebook.kernel;\n            var comm_manager = kernel.comm_manager;\n\n            //attempt to retrieve an already open connection with the same id.\n            var comm_id = event.data.comm_id;\n            var comm = comm_manager.comms[comm_id];\n\n            //if no communication was found, open it.\n            if(!comm) {\n                setupTreeCommunication(kernel, comm_id);\n            }\n            else {\n                // console.log(\"no setup required for \" + comm_id);\n            }\n\n\n            //try to access the comm again hoping the setup went well.\n            comm = comm_manager.comms[comm_id];\n\n            //if the setup is ok, send the message, otherwise queue it.\n            if (comm) {\n                comm.then(function (c) {\n                    // console.log(\"sending message to jl\");\n                    c.send(event.data.body);\n                });\n            }\n            else {\n                // console.log(\"could not open communication with id \" + comm_id);\n                window.DSWB.msgQueue.push(event.data.body);\n            }\n        }\n    }, false);\n\n    /*\n     * Attempts to open a new communication with unique id.\n     *\n     * @param {string} comm_id the id of the communication we are trying to open.\n     */\n    function setupTreeCommunication(kernel, comm_id) {\n        // console.log(\"entering setupTreeCommunication\");\n        if (window.DSWB.initComm) {\n            //setup js side\n            window.DSWB.initComm(undefined, IPython.notebook);\n\n            //setup jl side. Note that if a variabl auto_comm exists in the global julia scope, it'll get overwritten.\n            kernel.execute('auto_comm = Comm(:TreeWidget, \"' + comm_id + '\", true)');\n            kernel.execute(\"auto_comm.on_msg = DSWBIJuliaComm.receiverFactory(auto_comm)\");\n            kernel.execute(\"auto_comm = 0\");\n        } else {\n            // console.log(\"window doesn't hold a DSWB object\");\n        }\n    }\n\n    /*\n     * Return true if all the IJulia javascript variables are set.\n     */\n    function isIJuliaBroken() {\n        return !IPython || !IPython.notebook || !IPython.notebook.kernel || !IPython.notebook.kernel.comm_manager;\n    }\n\n}());\n </script>")

dsn set to vfdsn
beacons set to anonymous_beacons
resources set to anonymous_beacons_rt
startTime set to 2016-02-16T00:00:00
endTime set to 2016-02-16T23:59:00
customer set to Anonymous Retailer
productPageGroup set to Product
timezone set to America/Los_Angeles
topPageGroup set to Category
recipients set to ASCIIString["mostenberg@soasta.com"]


In [2]:
#CONNECT
tic()
setRedshiftEndpoint("$(dsn)")
tableName = setTable("$(beacons)", tableType="BEACON_TABLE")
setConversionMetric("custom_metrics_$(conversionMetric)")
toc()

Redshift endpoint set: 2017-02-17T16:55:44
elapsed time: 0.541521989 seconds


0.541521989

In [3]:
#CONFIGURE
if (Base.length(timezone)==0) timezone="America/Los_Angeles" end #Set timezone to West Coast if not already set. Other options: "America/Chicago", "America/New_York"
if (startTime=="") startTime=getBeaconsFirstAndLast()[:1] ; print("startTime set to $(startTime)\t") else startTime = datetimeToUTC(startTime, TimeZone(timezone)) end #Set start time to first beacon if not set.
if (endTime=="") endTime=getBeaconsFirstAndLast()[:2] ; print("endTime set to $(endTime)\t") else endTime = datetimeToUTC(endTime, TimeZone(timezone)); end #Set end time to last beacon if not set.
fieldNames = [ :user_agent_device_type, :page_group, :user_agent_family ]

3-element Array{Symbol,1}:
 :user_agent_device_type
 :page_group            
 :user_agent_family     

In [4]:
startTimeMs=datetimeToMs(datetimeToUTC(startTime,TimeZone(timezone)))
endTimeMs=datetimeToMs(datetimeToUTC(endTime,TimeZone(timezone)))
treemapView = "$(tableName)_shortdates"
select("""create or replace view $treemapView as (
select *
from $tableName 
where timestamp between $(startTimeMs) and $(endTimeMs)
)""")
t = getTreemapData(treemapView, fieldNames, forceRefresh=true)
overallMedian = getOverallMedianLoadTime(t)
select("""drop view $(treemapView)""")

INFO:  Relation "tree_schema" already exists and will be skipped
INFO:  Schema "tree_data_b4cd4347_a75e_40c4_abb2_efc4b015c9e9" does not exist and will be skipped
INFO:  Table "anonymous_beacons_shortdates" does not exist and will be skipped
INFO:  Table "anonymous_beacons_shortdates_user_agent_device_type" does not exist and will be skipped
INFO:  Table "anonymous_beacons_shortdates_page_group" does not exist and will be skipped
INFO:  Table "anonymous_beacons_shortdates_user_agent_family" does not exist and will be skipped


In [5]:
displayTitle(chart_title = "Drillable Treemap",chart_info=["$(customer) data from $(startTime) to $endTime"],showTimeStamp = false)
drawTreev2(t, "$(customer)", overallMedian)

Drillable Treemap Anonymous Retailer data from 2016-02-16T08:00:00 to 2016-02-17T07:59:00

HTML{ASCIIString}("    <script>\n    (function () {\n    window.DSWB = window.DSWB || {};\n    //attach initComm function to window\n    window.DSWB.initComm = function(evt, data) {\n        // console.log(\"entering initComm\");\n        var comm_manager = data.kernel.comm_manager;\n        comm_manager.register_target(\"TreeWidget\", treeReceiver);\n    }\n\n    try {\n        window.DSWB.initComm(undefined, IPython.notebook);\n    }\n    catch (e) {\n        window.addEventListener(\"kernel_created.Kernel kernel_created.Session\", window.DSWB.initComm, false);\n    }\n\n    function treeReceiver(comm, comm_open) {\n        //If there are queued messages when the communication opens, send them.\n        if (window.DSWB.msgQueue && window.DSWB.msgQueue.length) {\n            var msg;\n            for (var i = 0; i < window.DSWB.msgQueue.length; i++) {\n                msg = window.DSWB.msgQueue[i];\n                comm.send(msg);\n            }\n            //empty the queue\n            window.DSWB.msgQueue = [];\n        }\n\n        comm.on_msg(function(msg) {\n            var content = msg.content;\n            var data = content.data;\n            var comm_id = content.comm_id;\n            var forwarded = {};\n            var target = window.location.protocol + \"//\" + window.location.host;\n            forwarded.type = data.type;\n            forwarded.msg_id = msg.msg_id;\n            forwarded.detail = {\n                treeId: comm_id,\n                treeData: data\n            };\n            //if an iframe exists on the page with the ID of the message's communication, post to the iframe, otherwise, post to the window for now.\n            if (document.getElementById(comm_id) && document.getElementById(comm_id).contentWindow) {\n                document.getElementById(comm_id).contentWindow.postMessage(forwarded, target);\n            }\n            else {\n                window.postMessage(forwarded, target);\n            }\n        });\n\n    }\n\n}());\n\n\n    </script>\n")

HTML{ASCIIString}("<iframe id=\"1bfa0f47-7567-4200-b1a2-e93a12dee523\" width=\"100%\"  src=\"about:blank\" scrolling=\"auto\"></iframe>\n<script>\n    (function() {\n        var iframe = document.getElementById(\"1bfa0f47-7567-4200-b1a2-e93a12dee523\");\n        iframe.treeLoaded = function(runner, id) {\n            iframe.runner = runner;\n        };\n        iframe.height = document.documentElement.clientHeight / 1.3;\n        iframe.src = \"/static/graphs/templates/treetemplate_v2.html#id=1bfa0f47-7567-4200-b1a2-e93a12dee523\";\n        iframe.onload = function() {\n            iframe.contentWindow.postMessage({\"detail\":{\"treeData\":{\"treeData\":{\"data\":[{\"start_time\":0,\"end_time\":0,\"Beacons\":11858,\"DOM Size (byte)\":460876,\"id\":4,\"Load Time (s)\":4.7935},{\"start_time\":0,\"end_time\":0,\"Beacons\":100465,\"DOM Size (byte)\":458436,\"id\":2,\"Load Time (s)\":5.707},{\"start_time\":0,\"end_time\":0,\"Beacons\":266826,\"DOM Size (byte)\":455420,\"id\":3,\"Load Time (s)\":4.811},{\"start_time\":0,\"end_time\":0,\"Beacons\":549719,\"DOM Size (byte)\":454364,\"id\":1,\"Load Time (s)\":2.721}],\"title\":\"Anonymous Retailer\"},\"currentField\":\"user_agent_device_type\",\"treeType\":\"beacon\",\"allFields\":[\"user_agent_device_type\",\"page_group\",\"user_agent_family\"],\"aliases\":{\"user_agent_device_type\":{\"4\":\"Not Available\",\"2\":\"Tablet\",\"3\":\"Mobile\",\"1\":\"Desktop\"},\"user_agent_family\":{\"2\":\"Chrome Mobile\",\"11\":\"Iron\",\"39\":\"Pale Moon (Firefox Variant)\",\"25\":\"QQ Browser Mobile\",\"42\":\"moatbot\",\"29\":\"AOL\",\"8\":\"Maxthon\",\"20\":\"IE\",\"31\":\"BlackBerry WebKit\",\"14\":\"BlackBerry\",\"33\":\"NetFront NX\",\"18\":\"Safari\",\"26\":\"AppleMail\",\"35\":\"(Unknown)\",\"17\":\"Chrome\",\"44\":\"Opera Mini\",\"37\":\"Nokia Services (WAP) Browser\",\"4\":\"Amazon Silk\",\"13\":\"Firefox Alpha\",\"30\":\"Chromium\",\"1\":\"Edge\",\"32\":\"UC Browser\",\"40\":\"Firefox Beta\",\"7\":\"IE Mobile\",\"9\":\"Chrome Frame\",\"43\":\"Avant\",\"34\":\"SeaMonkey\",\"3\":\"Android Browser\",\"38\":\"Puffin\",\"36\":\"PhantomJS\",\"12\":\"Yandex Browser\",\"16\":\"Sogou Explorer\",\"21\":\"Firefox Mobile\",\"10\":\"Nokia Browser\",\"19\":\"Mobile Safari\",\"22\":\"Opera\",\"6\":\"Halebot\",\"24\":\"Opera Mobile\",\"28\":\"Opera Coast\",\"5\":\"Chrome Mobile iOS\",\"23\":\"Other\",\"27\":\"Firefox\",\"15\":\"Iceweasel\",\"41\":\"Genieo\"},\"page_group\":{\"68\":\"CVAddressAdd\",\"2\":\"ForgotPasswordView\",\"11\":\"ReviewForm\",\"39\":\"Homepage\",\"46\":\"ContactUs\",\"25\":\"ForgotPassword\",\"55\":\"PrintEcardDisplay\",\"42\":\"StoreLocator\",\"29\":\"Home\",\"58\":\"cHome\",\"66\":\"TopCategoriesDisplay\",\"59\":\"OrderShipping\",\"8\":\"UserRegistrationForm\",\"57\":\"Category\",\"20\":\"OrderAddress\",\"31\":\"ChangePassword\",\"14\":\"OrderItemUpdate\",\"33\":\"AddressBookForm\",\"18\":\"BrandPage\",\"52\":\"CategoryDisplay\",\"69\":\"OrderItemMove\",\"26\":\"OrderReturnProcess\",\"35\":\"MultipleOrderProfileUpdate\",\"17\":\"FiftyOneResultView\",\"65\":\"OrderReturnDisplay\",\"64\":\"RememberMeLogon\",\"49\":\"ReLogonFormView\",\"44\":\"ManageEcardsDisplay\",\"37\":\"FiftyOneXML\",\"4\":\"OrderReview\",\"45\":\"OrderHistoryList\",\"13\":\"OrderProcess\",\"67\":\"OrderItemAdd\",\"30\":\"OrderReturnLabelReprint\",\"1\":\"ContextChooserView\",\"47\":\"GiftCardBalanceLookupView\",\"54\":\"OrderCalculate\",\"50\":\"Logout\",\"32\":\"InterestItemDelete\",\"40\":\"StoreLocatorDetail\",\"7\":\"OrderPayment\",\"9\":\"TemporaryPasswordSentForm\",\"43\":\"MysiteFrameView\",\"60\":\"Wishlist\",\"34\":\"AddressForm\",\"3\":\"ResetPassword\",\"61\":\"OrderReturnAddressProcess\",\"38\":\"Shopping Cart\",\"36\":\"Product\",\"48\":\"OrderReviewCmd\",\"12\":\"RPMCodeValidateCmd\",\"16\":\"Search Results\",\"62\":\"ResetPasswordRecommendView\",\"21\":\"OrderConfirmation\",\"10\":\"UserRegistrationUpdate\",\"19\":\"(No Page Group)\",\"51\":\"OrderItemDelete\",\"22\":\"UserRegistrationAdd\",\"6\"